# Imports

In [1]:
!pip install python-dotenv

In [2]:
import requests
from kafka import KafkaProducer
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

In [3]:
# Charger les variables d'environnement depuis le fichier .env copié dans le conteneur
load_dotenv('/home/jovyan/.env')

# Configuration de l'API OpenSky
OPENSKY_ALL_FLIGHTS_URL = "https://opensky-network.org/api/flights/all?begin={}&end={}"
USERNAME = os.environ.get('OPENSKY_USERNAME')
PASSWORD = os.environ.get('OPENSKY_PASSWORD')

[{'icao24': '84630c', 'firstSeen': 1517229441, 'estDepartureAirport': None, 'lastSeen': 1517230681, 'estArrivalAirport': None, 'callsign': 'JJP120  ', 'estDepartureAirportHorizDistance': None, 'estDepartureAirportVertDistance': None, 'estArrivalAirportHorizDistance': None, 'estArrivalAirportVertDistance': None, 'departureAirportCandidatesCount': 0, 'arrivalAirportCandidatesCount': 0}, {'icao24': '800547', 'firstSeen': 1517227477, 'estDepartureAirport': 'VGTJ', 'lastSeen': 1517228618, 'estArrivalAirport': None, 'callsign': 'JAI273  ', 'estDepartureAirportHorizDistance': 6958, 'estDepartureAirportVertDistance': 655, 'estArrivalAirportHorizDistance': None, 'estArrivalAirportVertDistance': None, 'departureAirportCandidatesCount': 1, 'arrivalAirportCandidatesCount': 0}, {'icao24': 'a403ca', 'firstSeen': 1517228212, 'estDepartureAirport': None, 'lastSeen': 1517229651, 'estArrivalAirport': None, 'callsign': None, 'estDepartureAirportHorizDistance': None, 'estDepartureAirportVertDistance': Non

# Fonction pour envoyer les données OpenSky à Kafka

In [4]:

def send_opensky_to_kafka(topic, url, username, password):
    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from OpenSky
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        data = response.json()
        if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
            flights = [item for item in data if 'icao24' in item]
        else:
            flights = []


        # Publish each state to Kafka
        for flight in flights:
            producer.send(topic, value=flight)
            #print(f"Sent: {flight}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(flights)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [5]:
flights = {
    "icao24": "icao24",
    "firstSeen": "firstSeen",
    "estDepartureAirport": "estDepartureAirport",
    "lastSeen": "lastSeen",
    "estArrivalAirport": "estArrivalAirport",
    "estDepartureAirportHorizDistance": "estDepartureAirportHorizDistance",
    "estDepartureAirportVertDistance": "estDepartureAirportVertDistance",
    "estArrivalAirportHorizDistance": "estArrivalAirportHorizDistance",
    "estArrivalAirportVertDistance": "estArrivalAirportVertDistance",
    "departureAirportCandidatesCount": "departureAirportCandidatesCount",
    "arrivalAirportCandidatesCount": "arrivalAirportCandidatesCount",
}

# Durée de récupération (minutes)
step = 60

# date initiale
initial_date_str = "2025-01-25 16:00:00"

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)


start = int(initial_date.timestamp())
end = int(end_date.timestamp())




In [6]:
# Send OpenSky data to Kafka

for i in range(10):
    print(f"{start}-{end}")
    send_opensky_to_kafka("opensky-flights", OPENSKY_ALL_FLIGHTS_URL.format(start, end), USERNAME, PASSWORD)
    time.sleep(5)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = int(initial_date.timestamp())
    end = int(end_date.timestamp())

1737820800-1737824400


AttributeError: 'list' object has no attribute 'get'